In [1]:
import torch
import torch.nn as nn

import torch_sparse
import torch_geometric.transforms as T

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

In [2]:
class Conv(nn.Module):
    def __init__(self, input_channels, out_channels):
        super(Conv, self).__init__()

        self.weight = nn.Parameter(torch.Tensor(input_channels, out_channels))
        self.bias = nn.Parameter(torch.Tensor(out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)
        
    def forward(self, x, adj):
        x = torch.matmul(x, self.weight)
        out = torch_sparse.matmul(adj, x) + self.bias
        return out

class GCN(nn.Module):
    def __init__(self, input_channels, out_channels, hidden_channels, num_layers, p):
        super(GCN, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(Conv(input_channels, hidden_channels))
        
        self.bns = nn.ModuleList()
        self.bns.append(nn.BatchNorm1d(hidden_channels))
        
        for _ in range(num_layers - 2):
            self.convs.append(Conv(hidden_channels, hidden_channels))
            self.bns.append(nn.BatchNorm1d(hidden_channels))
            
        self.convs.append(Conv(hidden_channels, out_channels))
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p)
        self.num_layers = num_layers
        
    def forward(self, x, adj):
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, adj)
            x = self.bns[i](x)
            x = self.relu(x)
            x = self.dropout(x)
        out = self.convs[-1](x, adj)
        return out

def train(model, data, train_idx, optimizer, criterion):
    model.train()
    
    model.zero_grad()
    outputs = model(data.x, data.adj_t)[train_idx]
    loss = criterion(outputs, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()
    
    outputs = model(data.x, data.adj_t)
    y_pred = outputs.argmax(dim=1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

hidden_channels = 256
num_layers = 3
dropout = 0.5
epochs = 500
log_steps = 10
lr = 0.01

In [4]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', root='../dataset', transform=T.ToSparseTensor())

data = dataset[0]
data.adj_t = data.adj_t.to_symmetric()
data = data.to(device)

In [5]:
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [6]:
model = GCN(data.num_features, dataset.num_classes, hidden_channels, num_layers, dropout).to(device)

In [7]:
evaluator = Evaluator(name='ogbn-arxiv')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [8]:
data.adj_t = torch_sparse.fill_diag(data.adj_t, 1)
deg = torch_sparse.sum(data.adj_t, 0).pow_(-0.5)
data.adj_t = torch_sparse.mul(data.adj_t, deg.view(-1, 1))
data.adj_t = torch_sparse.mul(data.adj_t, deg.view(1, -1))

test_scores = []
for epoch in range(1, 1 + epochs):
    loss = train(model, data, train_idx, optimizer, criterion)
    result = test(model, data, split_idx, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        test_scores.append(test_acc)
        print(f'Run: {1:02d}, '
              f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')
print(f"Best test accuracy: {max(test_scores) * 100:.2f}%")

Run: 01, Epoch: 10, Loss: 1.3851, Train: 35.37%, Valid: 34.71% Test: 39.87%
Run: 01, Epoch: 20, Loss: 1.1905, Train: 52.70%, Valid: 53.25% Test: 56.23%
Run: 01, Epoch: 30, Loss: 1.1039, Train: 67.09%, Valid: 67.37% Test: 66.99%
Run: 01, Epoch: 40, Loss: 1.0493, Train: 69.70%, Valid: 69.67% Test: 69.31%
Run: 01, Epoch: 50, Loss: 1.0107, Train: 70.95%, Valid: 70.57% Test: 69.89%
Run: 01, Epoch: 60, Loss: 0.9852, Train: 71.93%, Valid: 70.49% Test: 68.60%
Run: 01, Epoch: 70, Loss: 0.9617, Train: 72.53%, Valid: 71.43% Test: 70.59%
Run: 01, Epoch: 80, Loss: 0.9429, Train: 72.91%, Valid: 70.90% Test: 69.23%
Run: 01, Epoch: 90, Loss: 0.9266, Train: 73.33%, Valid: 71.32% Test: 69.73%
Run: 01, Epoch: 100, Loss: 0.9112, Train: 73.71%, Valid: 72.08% Test: 71.25%
Run: 01, Epoch: 110, Loss: 0.8974, Train: 74.20%, Valid: 72.21% Test: 70.75%
Run: 01, Epoch: 120, Loss: 0.8858, Train: 74.60%, Valid: 72.03% Test: 71.04%
Run: 01, Epoch: 130, Loss: 0.8737, Train: 74.90%, Valid: 71.77% Test: 70.00%
Run: 01,

In [9]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

110120
